In [9]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
import matplotlib.pyplot as plt
#from SimpleCV import Image
#from SimpleCV import Camera
os.environ['OPENCV_VIDEOIO_DEBUG'] = '1'

path = 'ImagesAttendance'
images = []
classNames = []
myList = os.listdir(path)
print(myList)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)


['Huy_Tan.jpg', 'Nam_Tran.jpg', 'Elon_Musk.jpg', 'ronaldo.jpg', 'Messi.jpg']
['Huy_Tan', 'Nam_Tran', 'Elon_Musk', 'ronaldo', 'Messi']


In [10]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faceLocs = face_recognition.face_locations(img)
        if len(faceLocs) == 0:
            continue
        encode = face_recognition.face_encodings(img, faceLocs)[0]
        encodeList.append(encode)
    return encodeList


In [11]:
def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.write(f'\n{name},{dtString}')


In [12]:
encodeListKnown = findEncodings(images)
print('Encoding Complete')
cap = cv2.VideoCapture(0)


Encoding Complete


[ WARN:0@138.427] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@138.428] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


In [13]:
while True:
    success, img = cap.read()
    
    if not success:
        print("Không thể đọc khung hình từ camera")
        break
    
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)
    
    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        matchIndex = np.argmin(faceDis)
        
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            markAttendance(name)
        else:
            name = 'Unknown'
            
        y1, x2, y2, x1 = faceLoc
        y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.rectangle(img, (x1, y2-35), (x2, y2), (0, 0, 255), cv2.FILLED)
        cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
        
    cv2.imshow('Webcam', img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Không thể đọc khung hình từ camera
